<center>

<h1 style="text-align:center"> (Side) Effects </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Why Side Effects

* We have only used **purely functional** feature of OCaml
* Our study of lambda calculus used only **purely functional** features

* The above statements are *lies*.
  + We have used `print_endline`, `printf` and other features to display our results to screen.
* It is sometimes useful to write programs that have **side effects**

## Side effects

Side effects include

* Mutating (i.e., destructively updating) the values of program state. 
* Reading from standard input, printing to standard output.
* Reading and writing to files, sockets, pipes etc.
* ...
* Composing, sending and receiving emails, editing documents, writing this slide, etc.

## Side effects in OCaml

* OCaml programs can include side effects
* Features
  + Mutations: Reference cells, Arrays, Mutable record fields
  + I/O of all sorts
* In this lecture, **Mutations**

$
\require{color}
\newcommand{\cpurple}[1]{{\color{\purple}{\text{#1}}}}
\newcommand{\cred}[1]{{\color{\red}{\text{#1}}}}
$

## Reference cells

* Aka "refs" or "ref cell"
* Pointer to a typed location in memory
* The binding of a variable to a ref cell is $\cpurple{immutable}$ 
  + but the contents of the ref cell may $\cred{change}$.

## Reference cells

In [32]:
let r = ref 0

val r : int ref = {contents = 0}


In [33]:
r := !r + 1;
!r

- : int = 1


## Reference Cells: Types

In [34]:
ref

- : 'a -> 'a ref = <fun>


In [35]:
(!)

- : 'a ref -> 'a = <fun>


In [37]:
(:=)

- : 'a ref -> 'a -> unit = <fun>


## Implementing a counter

In [47]:
let make_counter init =
  let c = ref init in
  fun () -> 
    (c := !c + 1; !c)

val make_counter : int -> unit -> int = <fun>


In [48]:
let next = make_counter 0

val next : unit -> int = <fun>


In [56]:
next()

- : int = 8


## Side effects make reasoning hard

* Recall that referential transparency allows replacing $e$ with $v$ if $e \rightarrow_{\beta} v$.
* Side effects break referential transparency.

## Referential transparency

Consider the function `foo`:

In [57]:
let foo x = x + 1

val foo : int -> int = <fun>


In [58]:
let baz = foo 10

val baz : int = 11


`baz` can now be optimised to

In [ ]:
let baz = 11

## Referential transparency

Consider the function `bar`:

In [60]:
let bar x = x + next()

val bar : int -> int = <fun>


In [62]:
let qux = bar 10

val qux : int = 20


Can we now optimise `qux` to:

In [ ]:
let qux = 19

**NO**. Referential transparency breaks under side effects. 

## Aliases

References may create aliases.

What is the result of this program?

In [63]:
let x = ref 10 in
let y = ref 10 in
let z = x in
z := !x + 1;
!x + !y

- : int = 21


* `z` and `x` are said to be **aliases**
  + They refer to the same object in the program heap.

## Equality

* The `=` that we have been using is known as **structural equality**
  + Checks whether the values' structurally equal. 
  + `[1;2;3] = [1;2;3]` evaluates to `true`. 
* Because of references, one may also want to ask whether two expressions are **aliases**
  + This equality is known as **physical equality**.
  + OCaml uses `==` to check for physical equality.

## Equality

In [ ]:
let l1 = [1;2;3];;
let l2 = l1;;
let l3 = [1;2;3];;
let r1 = ref l1;;
let r2 = r1;;
let r3 = ref l3;;

## Equality
```ocaml
let l1 = [1;2;3];; 
let l2 = l1;; 
let l3 = [1;2;3];;
let r1 = ref l1;;
let r2 = r1;;
let r3 = ref l3;;
```
which of the following are true?

(1)`l1 = l2` (2)`l1 = l3` (3)`r1 == r2` (4)`l1 == l2` 

(5) `r1 == r3` (6) `l1 == l3` (7) `r1 = r2` (8) `r1 = r3`

## Equality

<center>
    
<img src="heap.svg" width="250">
</center>

which of the following are true?

(1)`l1 = l2` (2)`l1 = l3` (3)`r1 == r2` (4)`l1 == l2` 

(5) `r1 == r3` (6) `l1 == l3` (7) `r1 = r2` (8) `r1 = r3`

References are structurally equal iff their contents are structurally equal.

In [ ]:
l1 = l2

## Value Restriction

Consider the following program:

In [64]:
let r = [] in
let r1 : int list = r in
let r2 : string list = r in
(r1,r2)

- : int list * string list = ([], [])


`r` has type `'a list`. But otherwise, nothing surprising here.

## Value Restriction

Consider a modified program:

In [65]:
let r = ref [] in
let r1 : int list ref = r in
let r2 : string list ref = r in
(r1,r2)

error: compile_error

## Value Restriction

Let's look at the type of `ref []`

In [66]:
let r = ref []

val r : '_weak11 list ref = {contents = []}


* The `'_weak1` says that `r` is only *weakly polymorphic*. 
  + `r` can be used with only one type.
  + This is known as **value restriction**.
* But why does value restriction exist?

## Why does value restriction exist?

If value restriction does not exist, the following program would be well-typed.


```ocaml
let r = ref [] in
let r1 : int list ref = r in
let r2 : string list ref = r in
r1 := [1];
print_endline (List.hd !r2)
```

* We are storing an int list in `r1` and reading it out as a string list through `r2`.
* In OCaml, value restriction is implemented as a syntactic check of RHS + some typing checks.
  + Details are beyond the scope of this course. 

## Partial Application and Value restriction 

Since value restriction is implemented as a syntactic check, it can sometimes be restrictive. 

For example, here is a function that swaps the elemenents of a pair in a list of pairs.

In [67]:
let swap_list = List.map (fun (a,b) -> (b,a))

val swap_list : ('_weak12 * '_weak13) list -> ('_weak13 * '_weak12) list =
  <fun>


The type inferred is a weakly polymorphic type.

In [68]:
(swap_list [(1,"hello")], 
 swap_list [(1,1)])

error: compile_error

## Partial Application and Value restriction 

In many cases, the unnecessary value restriction can be fixed by eta expansion. 

In [69]:
let swap_list l = List.map (fun (a,b) -> (b,a)) l

val swap_list : ('a * 'b) list -> ('b * 'a) list = <fun>


In [70]:
(swap_list [(1,"hello")], 
 swap_list [(1,1)])

- : (string * int) list * (int * int) list = ([("hello", 1)], [(1, 1)])


## Mutable Record Fields

Ref cells are essentially syntactic sugar:

```ocaml
type 'a ref = { mutable contents: 'a }
let ref x = { contents = x }
let ( ! ) r = r.contents
let ( := ) r newval = r.contents <- newval
```

* That type is declared in `Pervasives`
* The functions are compiled down to something equivalent

## Doubly-linked list

In [72]:
(* The type of elements *)
type 'a element = { 
  value : 'a;
  mutable next : 'a element option;
  mutable prev : 'a element option
}

(* The type of list *)
type 'a dllist = 'a element option ref

type 'a element = {
  value : 'a;
  mutable next : 'a element option;
  mutable prev : 'a element option;
}


type 'a dllist = 'a element option ref


## Double-linked list

In [73]:
let create () : 'a dllist = ref None
let is_empty (t : 'a dllist) = !t = None

let value elt = elt.value

let first (t : 'a dllist) = !t
let next elt = elt.next
let prev elt = elt.prev

val create : unit -> 'a dllist = <fun>


val is_empty : 'a dllist -> bool = <fun>


val value : 'a element -> 'a = <fun>


val first : 'a dllist -> 'a element option = <fun>


val next : 'a element -> 'a element option = <fun>


val prev : 'a element -> 'a element option = <fun>


## Doubly-linked list

In [77]:
let insert_first (t:'a dllist) value =
  let new_elt = { prev = None; next = !t; value } in
  begin match !t with
  | Some old_first -> old_first.prev <- Some new_elt
  | None -> ()
  end;
  t := Some new_elt;
  new_elt

val insert_first : 'a dllist -> 'a -> 'a element = <fun>


## Doubly-linked list

In [76]:
let insert_after elt value =
  let new_elt = { value; prev = Some elt; next = elt.next } in
  begin match elt.next with
  | Some old_next -> old_next.prev <- Some new_elt
  | None -> ()
  end;
  elt.next <- Some new_elt;
  new_elt

val insert_after : 'a element -> 'a -> 'a element = <fun>


## Doubly-linked list

In [ ]:
let remove (t:'a dllist) elt =
  let { prev; next; _ } = elt in
  begin match prev with
  | Some prev -> prev.next <- next
  | None -> t := next
  end;
  begin match next with
  | Some next -> next.prev <- prev;
  | None -> ()
  end;
  elt.prev <- None;
  elt.next <- None

## Doubly-linked list

In [74]:
let iter (t : 'a dllist) f =
  let rec loop = function
    | None -> ()
    | Some el -> f (value el); loop (next el)
  in
  loop !t

val iter : 'a dllist -> ('a -> 'b) -> unit = <fun>


## Doubly-linked list

In [78]:
let l = create ();;
let n0 = insert_first l 0;;
let n1 = insert_after n0 1;;
insert_after n1 2

val l : '_weak15 dllist = {contents = None}


val n0 : int element = {value = 0; next = None; prev = None}


val n1 : int element =
  {value = 1; next = None;
   prev = Some {value = 0; next = Some <cycle>; prev = None}}


- : int element =
{value = 2; next = None;
 prev =
  Some
   {value = 1; next = Some <cycle>;
    prev = Some {value = 0; next = Some <cycle>; prev = None}}}


## Doubly-linked list

In [79]:
iter l (Printf.printf "%d\n%!")

0
1
2


- : unit = ()


## Arrays

Collection type with efficient random access.

In [ ]:
let a = [| 1;2;3 |]

In [ ]:
a.(2)

## Arrays

In [ ]:
a.(1) <- 0; 
a

In [ ]:
a.(4)

* Arrays behave very similar to array's in C
 + See [Array module](https://caml.inria.fr/pub/docs/manual-ocaml/libref/Array.html) documentation in the OCaml standard library

## Benefits of immutability

* Programmer doesn’t have to think about aliasing; can concentrate on other aspects of code
* Language implementation is free to share objects on the heap, which is cheap
* Often easier to reason about whether code is correct
* Perfect fit for concurrent programming

But

* Some data structures (hash tables, arrays, ...) are more efficient if imperative

<center>

<h1 style="text-align:center"> Fin. </h1>
</center>